<img src="../Pics/MLSb-T.png" width="160">
<br><br>
<center><u><H1>CNN on CIFAR10 with Keras</H1></u></center>

In [ ]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = True
sess = tf.Session(config=config)
set_session(sess)

In [ ]:
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
(X_train_, y_train), (X_test_, y_test) = cifar10.load_data()

In [ ]:
print('X_train shape:', X_train_.shape)
print('train samples: ',X_train_.shape[0])
print('test samples: ', X_test_.shape[0])

In [ ]:
num_train, img_rows, img_cols, img_channels =  X_train_.shape

In [ ]:
# image dimension:
input_shape = (img_rows, img_cols, 3)

In [ ]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [ ]:
n_classes = len(np.unique(y_train))
n_classes

In [ ]:
classes = np.unique(y_train)
classes

## Visualizing the classes with an image example

In [ ]:
fig = plt.figure(figsize=(10,6))
for i in range(n_classes):
    ax = fig.add_subplot(2, 5, 1 + i, xticks=[], yticks=[])
    idx = np.where(y_train[:]==i)[0]
    features_idx = X_train_[idx,::]
    img_num = np.random.randint(features_idx.shape[0])
    image = features_idx[img_num,::].reshape((32, 32, 3))
    ax.set_title(class_names[i])
    plt.imshow(image)
plt.show()

In [ ]:
#Normalize the input data
X_train = X_train_.astype('float32')/255.
X_test = X_test_.astype('float32')/255.

In [ ]:
#One-hot encode the labels
y_train = np_utils.to_categorical(y_train, n_classes)
y_test = np_utils.to_categorical(y_test, n_classes)

## CNN architecture

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(32, kernel_size=(3, 3), input_shape= input_shape, activation = 'relu'))
model.add(BatchNormalization())
model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2), padding='same'))
model.add(Dropout(0.3))

In [ ]:
model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation = 'relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2), padding='same'))
model.add(Dropout(0.3))

In [ ]:
model.add(Conv2D(128, kernel_size=(3, 3), padding='same', activation = 'relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=(3, 3), padding='same', activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2), padding='same'))
model.add(Dropout(0.5))

In [ ]:
model.add(Flatten())
model.add(Dense(units = 512, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.50))
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(units = n_classes, activation = 'softmax'))

In [ ]:
print(model.summary())

## Compiling the model:

In [ ]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [ ]:
# Define a callback to prevent overfitting:
callbacks = [EarlyStopping(monitor='val_acc', patience=5)]
#monitor: quantity to be monitored
#patience: number of epochs with no improvement after which training will be stopped.

In [ ]:
#hyperparameters
batch_size = 128
n_epochs = 20

In [ ]:
results = model.fit(X_train, y_train, batch_size=batch_size, epochs=n_epochs,
          verbose=1, validation_data=(X_test, y_test),callbacks=callbacks)

## Saving the model:

In [ ]:
model.save_weights('../data/cifar10_weights.h5', overwrite=True)

In [ ]:
model.save('../data/cifar10.h5')

In [ ]:
from keras.models import load_model

In [ ]:
loaded_model = load_model('../data/cifar10.h5')

## References:

https://www.cs.toronto.edu/~kriz/cifar.html

http://pillow.readthedocs.io/en/3.0.x/reference/Image.html?highlight=from%20array#PIL.Image.fromarray

https://keras.io/callbacks/